Sources

#### Projet End-to-End (Pas NHL API)
https://medium.com/coinmonks/4-718-using-machine-learning-to-bet-on-the-nhl-25d16649cd52

#### Projet Live Feed (NHL API)
##### Utilise le live feed pour shot sur gardien
https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d

#### Docu NHL API
https://github.com/dword4/nhlapi

Ancienne Doc:
https://webcache.googleusercontent.com/search?q=cache:ILYmzDuxW70J:https://github.com/dword4/nhlapi+&cd=1&hl=en&ct=clnk&gl=ca

In [16]:
# Check Python Version
from platform import python_version
import pandas as pd
import requests  
from pandas.io.json import json_normalize
import datetime

pd.set_option('display.max_columns', None)

print(pd.show_versions())


INSTALLED VERSIONS
------------------
commit           : None
python           : 3.6.5.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
machine          : AMD64
processor        : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.None

pandas           : 1.0.1
numpy            : 1.17.2
pytz             : 2019.3
dateutil         : 2.8.0
pip              : 19.3.1
setuptools       : 41.6.0.post20191030
Cython           : None
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 2.10.3
IPython          : 7.9.0
pandas_datareader: None
bs4              : 4.8.1
bottleneck       : None
fastparquet      : None
gcsfs            : None
lxml.etree       : None
matpl

### import game data

In [10]:
start_date = datetime.date(2019, 3, 1)
end_date = datetime.date(2019, 3, 31)
delta = datetime.timedelta(days=1)

data_list = []

while start_date <= end_date:
    
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/schedule?date=' + start_date.strftime("%Y-%m-%d"))
    data = r.json()
    
    df = pd.json_normalize(data = data['dates'], record_path='games', meta=['date'])
    
    data_list.append(df)
        
    start_date += delta

In [11]:
df = pd.concat(data_list, sort='False', ignore_index=True)

df.to_csv('../../data/games.csv')

### Import teams data

In [12]:
url = 'https://statsapi.web.nhl.com/api/v1/teams'
r = requests.get(url)
json_data = r.json()
teams = json_normalize(json_data['teams'], sep = "_")

teams.to_csv('../../data/teams.csv')

C:\Users\athompson-leduc\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.
